In [1]:
from bson import ObjectId
import folium
import numpy as np
from bson import json_util
import json
import googlemaps
import pandas as pd
# Librerías propias
import config
import location
import mongodb
import bbdd

Connected to mongodb in -> None


In [2]:
mongodb.getClient()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [88]:
db, coll = mongodb.getCollection('companies','crunch')

In [89]:
companies = list(coll.find({}))

# LOCATION FIELD

Create geolocation field for all offices in original collection 

In [8]:
for company in companies:
    if len(company['offices']) >= 1:
        for j in range(len(company['offices'])):
            office = company['offices'][j]
            if office['latitude'] is not None and office['longitude'] is not None:
                filtro = {'_id':ObjectId(company['_id'])}
                field = 'offices.'+str(j)+'.location'
                value = {'$set':{field:location.getLocationfor(office)}}
                coll.update_many(filtro, value)   

## Unwinding documents
- Get rid of all companies with no offices and all offices with no address

In [9]:
pipeline = [
    {
        '$project': {
            'name': 1, 
            'description': 1, 
            'founded_year': 1, 
            'deadpooled_year': 1, 
            'total_money_raised': 1, 
            'offices': 1
        }
    }, {
        '$unwind': {
            'path': '$offices', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$match': {
            'offices.latitude': {
                '$exists': True
            }, 
            'offices.address1': {
                '$ne': ''
            }
        }
    }
]

In [10]:
offices = list(coll.aggregate(pipeline))

# CITY CHOICE

In [13]:
data = pd.DataFrame(offices)

In [14]:
print(data.shape)
data.head(2)

(14528, 7)


,_id,name,founded_year,deadpooled_year,description,total_money_raised,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,2.0,Server Management Software,$0,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d90,Postini,1999.0,NaN,None,$0,"{'description': None, 'address1': '959 Skyway ..."


In [15]:
data.rename(columns={'description':'category'}, 
                 inplace=True)

In [16]:
df = pd.concat([data, data.offices.apply(pd.Series)], axis = 1)

In [17]:
df.head(2)

,_id,name,founded_year,deadpooled_year,category,total_money_raised,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,2.0,Server Management Software,$0,"{'description': 'Headquarters', 'address1': '4...",Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,52cdef7c4bab8bd675297d90,Postini,1999.0,NaN,None,$0,"{'description': None, 'address1': '959 Skyway ...",None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."


In [18]:
offices = df.drop(columns=["offices","location"])

In [19]:
offices.head(2)

,_id,name,founded_year,deadpooled_year,category,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,2.0,Server Management Software,$0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d90,Postini,1999.0,NaN,None,$0,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


## KPIS

Design KPIs that are going to help me choose in which country I wan't to set my company.
    - Age: average age of companies by city
    - Average money raised
    - has raised: Total companies that have raised money by city
    - % of companies that have raised money within the area

In [35]:
offices['age'] = offices.founded_year.apply(lambda x: (2019 - x))

In [36]:
offices.rename(columns={'description':'category',
                          'description':'office_desc'}, 
                 inplace=True)

In [37]:
offices.head()

,_id,name,founded_year,deadpooled_year,category,total_money_raised,office_desc,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,age,has_raised
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,2.0,Server Management Software,0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,23.0,0
1,52cdef7c4bab8bd675297d90,Postini,1999.0,NaN,None,0,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,20.0,0
2,52cdef7c4bab8bd675297d8f,Omnidrive,2005.0,2008.0,None,800000,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,14.0,1
3,52cdef7c4bab8bd675297d92,Flektor,NaN,NaN,None,0,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,NaN,0
4,52cdef7c4bab8bd675297d8c,Zoho,2005.0,3.0,Online Business Apps Suite,0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,14.0,0


In [ ]:
bbdd.quitaMonedas(offices,'total_money_raised')

In [39]:
offices['Average_money_raised']=offices['total_money_raised'].apply(bbdd.text_to_num)

TypeError: 'int' object is not subscriptable

In [25]:
offices['total_money_raised']

0             0
1             0
2        800000
3             0
4             0
          ...  
14523         0
14524         0
14525         0
14526         0
14527         0
Name: total_money_raised, Length: 14528, dtype: int64

In [26]:
offices['has_raised']=np.where(offices.total_money_raised>0,1,0)

In [40]:
co = offices[['country_code','city']].groupby(['city']).count()
avg_age = offices[['city','age']].groupby(['city']).agg({'age':'mean'})
avg_money_raised = offices[['city','total_money_raised']].groupby(['city']).agg({'total_money_raised':'mean'})
total_has_raised = offices[['city','has_raised']].groupby(['city']).agg({'has_raised':'sum'})

In [41]:
final = pd.concat([co, avg_age,total_has_raised, avg_money_raised], axis = 1).sort_values('country_code', ascending=False)


In [42]:
final['total_money_raised'] = final.total_money_raised/1000000
final['% Companies that has raised money'] = final.has_raised/final.country_code*100

In [45]:
final.rename(columns={'country_code':'Nº Companies',
                     'total_money_raised': 'Average money raised'}, 
                 inplace=True)

In [46]:
final[final.index!=''].head(10)

,Nº Companies,age,has_raised,Average money raised,% Companies that has raised money
city,,,,,
San Francisco,771,15.623704,432,23.044086,56.031128
New York,763,18.801802,356,19.828440,46.657929
London,551,16.374732,197,10.724397,35.753176
Seattle,191,14.782353,106,23.503738,55.497382
Austin,179,16.427632,88,22.138296,49.162011
Chicago,178,17.753247,58,23.085169,32.584270
Palo Alto,177,15.337838,110,36.448023,62.146893
Los Angeles,175,19.000000,55,6.487571,31.428571
San Jose,170,18.302817,78,26.981941,45.882353


## SELECTED CITY: AUSTIN

I select Austin to set up my gaming company. 
Studying the resulting collection I see that there are 38 offices that do not have neither latitude nor longitude populated. 
I use Google API to reverse geocode these offices.

In [102]:
db, austin = mongodb.getCollection('austin','crunch')

In [97]:
unlocated = list(austin.find({'offices.location':{'$exists':False}}))

In [ ]:
key = os.getenv('GOOGLE_KEY')
gmaps = googlemaps.Client(key=key)

In [ ]:
for co in unlocated:
    address_1 = (co['offices']['address1'],co['offices']['address2'],co['offices']['zip_code'],co['offices']['city'])
    address = ' '.join(address_1)
    filtro = {'_id':ObjectId(co['_id'])}
    field = 'offices.location'
    geo_result = gmaps.geocode(address)
    location = geo_result[0]['geometry']['location']
    lat = location['lat']
    lon = location['lng']
    loc = {
            'type':'Point',
            'coordinates':[float(lon), float(lat)]
            }
    value = {'$set':{field:loc}}
    austin.update_many(filtro, value)
    value2 ={'$set':{'offices.latitude':lat}}
    austin.update_many(filtro,value2)
    value3 ={'$set':{'offices.longitude':lon}}
    austin.update_many(filtro,value3)

# Nearby places collection

Next step is to create a new collection with all the different places/categories that I wan't my office to be close.
I use Foursquare API in order to achieve this goal

In [104]:
located = list(austin.find({}))

In [105]:
located[0]

{'_id': ObjectId('52cdef7c4bab8bd675297e4a'),
 'name': 'Indeed',
 'founded_year': 2004,
 'deadpooled_year': None,
 'description': 'Job Search Portal',
 'total_money_raised': '$5M',
 'offices': {'description': 'Headquarters',
  'address1': '7501 N. Capital of Texas Hwy',
  'address2': 'Building B',
  'zip_code': '78731',
  'city': 'Austin',
  'state_code': 'TX',
  'country_code': 'USA',
  'latitude': 30.3741311,
  'longitude': -97.7808373,
  'location': {'type': 'Point', 'coordinates': [-97.7808373, 30.3741311]}}}

In [ ]:
categoryId = '5744ccdfe4b0c0459246b4c7' #Guardería
radius = '1000'
category = 'Daycare'
daycare = location.getNearbyPlacesCat(located, categoryId, category, radius)

In [ ]:
categoryId = '4bf58dd8d48988d116941735,56aa371ce4b08b9a8d57356c,4bf58dd8d48988d117941735,4bf58dd8d48988d11e941735,4bf58dd8d48988d11b941735,50327c8591d4c4b30a586d5d,50327c8591d4c4b30a586d5d' #Bares
radius = '1000'
category = 'Bars'
bars = location.getNearbyPlacesCat(located, categoryId, category, radius)

In [ ]:
categoryId = '4bf58dd8d48988d1ed931735' #Aeropuerto
category = 'Airports'
radius = '20000'
airports = location.getNearbyPlacesCat(located, categoryId, category, radius)

In [ ]:
categoryId = '4bf58dd8d48988d1d3941735' #Vegetariano
category = 'Vegans'
radius = '1000'
vegans = location.getNearbyPlacesCat(located, categoryId, category, radius)

In [ ]:
query = 'Starbucks'
radius = '300'
category = 'Starbucks'
starbucks = location.getNearbyPlacesQuery(coll, query, category, radius)

In [ ]:
total = vegans + airports + daycare + bars + starbucks

In [ ]:
# Introducción del campo loc para aplicar geoindex en mongo
for place in total:
    place['loc'] = getLocationfor(place)

In [ ]:
with open('./input/total.json', 'w') as file:
     json.dump(total , file)

# Geospacial queries

In [3]:
mongodb.getClient()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
db, places = mongodb.getCollection('total','crunch')
db, austin = mongodb.getCollection('austin','crunch')

In [5]:
offices = list(austin.find({}))
nearby = list(places.find({}))

In [6]:
cat_dict = {'Daycare':600, 
            'Airports':25000,
            'Vegans': 1000, 
            'Starbucks':200,
            'Bars': 400}
ranking = []
rank_in = {}
for office in offices: 
    rank_in['_id'] = office['_id']
    rank_in['name'] = office['name']
    for cat in cat_dict:
        a = list(places.find({'$and':[{'location':{'$near':{'$geometry':office['offices']['location'],'$maxDistance':cat_dict[cat]}}},{'category':cat}]}))
        rank_in[cat] = len(a)
    ranking.append(rank_in)
    rank_in = {}
rkg = pd.DataFrame(ranking)

In [7]:
rkg.head()

,_id,name,Daycare,Airports,Vegans,Starbucks,Bars
0,52cdef7c4bab8bd675297e4a,Indeed,0,1,0,0,0
1,52cdef7c4bab8bd675297e32,Vast,0,1,2,1,13
2,52cdef7c4bab8bd675297fed,Myxer,0,1,0,0,1
3,52cdef7c4bab8bd675297dae,Thoof,0,1,3,0,7
4,52cdef7c4bab8bd675297f69,Spiceworks,0,1,0,0,1


In [8]:
rkg['selected']=rkg[['Daycare','Airports','Vegans','Starbucks', 'Bars']].apply(location.giveScore, axis = 1)

## Potential locations to set up the company are the following

In [9]:
rkg[rkg['selected']==1]

,_id,name,Daycare,Airports,Vegans,Starbucks,Bars,selected
135,52cdef7e4bab8bd67529b37e,Powered,1,1,2,1,12,1
155,52cdef7e4bab8bd67529bad2,AllClear ID,1,1,2,1,6,1


In [19]:
for i in rkg[rkg['selected']==1]['_id']:
    print(i)

52cdef7e4bab8bd67529b37e
52cdef7e4bab8bd67529bad2


In [26]:
list(austin.find({'_id':ObjectId('52cdef7e4bab8bd67529b37e')}))[0]

'Powered'

In [28]:
list(austin.find({'_id':ObjectId('52cdef7e4bab8bd67529b37e')}))[0]

{'_id': ObjectId('52cdef7e4bab8bd67529b37e'),
 'name': 'Powered',
 'founded_year': 1999,
 'deadpooled_year': None,
 'description': 'Leading Social Media Agency',
 'total_money_raised': '$19.4M',
 'offices': {'description': 'World Headquarters',
  'address1': '206 E. Ninth Street',
  'address2': '14th Floor',
  'zip_code': '78701',
  'city': 'Austin',
  'state_code': 'TX',
  'country_code': 'USA',
  'latitude': 30.2705775,
  'longitude': -97.7397293,
  'location': {'type': 'Point', 'coordinates': [-97.7397293, 30.2705775]}}}

In [98]:
cat_dict = {'Daycare':600, 
            'Airports':25000,
            'Vegans': 1000, 
            'Starbucks':200,
            'Bars': 400}

# map_city=folium.Map([30.266666, -97.733330], zoom_start = 100)
map_city=folium.Map(list(austin.find({'_id':ObjectId(selected)}))[0]['offices']['location']['coordinates'][::-1], zoom_start = 300)

for selected in rkg[rkg['selected']==1]['_id']:
    tooltip = list(austin.find({'_id':ObjectId(selected)}))[0]['name']
    folium.Marker(list(austin.find({'_id':ObjectId(selected)}))[0]['offices']['location']['coordinates'][::-1],
                radius=2,
                icon=folium.Icon(icon='home',color='black'), 
                tooltip = tooltip
               ).add_to(map_city)
    for cat in cat_dict:
        places_list = list(places.find({'$and':[{'location':{'$near':{'$geometry':list(austin.find({'_id':ObjectId(selected)}))[0]['offices']['location'],'$maxDistance':cat_dict[cat]}}},{'category':cat}]}))
        for place in places_list:
            tooltip = place['name']
            if place['category'] == 'Daycare':
                folium.Marker(place['location']['coordinates'][::-1],
                                    radius=2,
                                    icon=folium.Icon(icon='cloud',color='pink'), 
                                    tooltip = tooltip
                                       ).add_to(map_city)
            if place['category'] == 'Airports':
                folium.Marker(place['location']['coordinates'][::-1],
                                    radius=2,
                                    icon=folium.Icon(icon='plane',color='blue'), 
                                    tooltip = tooltip
                                       ).add_to(map_city)
            if place['category'] == 'Vegans':
                folium.Marker(place['location']['coordinates'][::-1],
                                    radius=2,
                                    icon=folium.Icon(icon='leaf',color='green'), 
                                    tooltip = tooltip
                                       ).add_to(map_city)
            if place['category'] == 'Bars':
                folium.Marker(place['location']['coordinates'][::-1],
                                    radius=2,
                                    icon=folium.Icon(icon='glass',color='purple'), 
                                    tooltip = tooltip
                                       ).add_to(map_city)
            if place['category'] == 'Starbucks':
                folium.Marker(place['location']['coordinates'][::-1],
                                    radius=2,
                                    icon=folium.Icon(icon='cloud',color='beige'), 
                                    tooltip = tooltip
                                       ).add_to(map_city)


In [99]:
map_city